# Data Preprocessing

Includes imputation and deletion of bad features.

### Import Dependencies

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', None)

### Load Data

In [3]:
train_df = pd.read_csv("project/data/train/features.csv", parse_dates = ['date_recorded' ],  na_values = [0, '0'])
labels = pd.read_csv("project/data/train/labels.csv")

test_df = pd.read_csv("project/data/test/test.csv", parse_dates = ['date_recorded' ],  na_values = [0, '0'])

# Merge features and labels files
train_df = pd.merge(labels, train_df, on='id')

print("Numerical columns: \n", train_df.select_dtypes(include=np.number).columns.tolist())

Numerical columns: 
 ['id', 'amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year']


In [4]:
train_df.head(5)

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390.0,Roman,34.938093,-9.856322,none,NaN,Lake Nyasa,Mnyusi B,Iringa,11,5.0,Ludewa,Mundindi,109.0,True,GeoData Consultants Ltd,VWC,Roman,False,1999.0,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,NaN,2013-03-06,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,NaN,Lake Victoria,Nyamara,Mara,20,2.0,Serengeti,Natta,280.0,NaN,GeoData Consultants Ltd,Other,NaN,True,2010.0,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,NaN,Pangani,Majengo,Manyara,21,4.0,Simanjiro,Ngorika,250.0,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009.0,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,NaN,2013-01-28,Unicef,263.0,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,NaN,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63.0,Nanyumbu,Nanyumbu,58.0,True,GeoData Consultants Ltd,VWC,NaN,True,1986.0,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,NaN,2011-07-13,Action In A,NaN,Artisan,31.130847,-1.825359,Shuleni,NaN,Lake Victoria,Kyanyamisa,Kagera,18,1.0,Karagwe,Nyakasimbi,NaN,True,GeoData Consultants Ltd,NaN,NaN,True,NaN,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


## 1. Data Imputation

In [5]:
missing = round((train_df.isna().sum())/len(train_df)*100,1)
missing = missing.sort_values(ascending = False)
missing.head(15)

num_private          98.7
amount_tsh           70.1
scheme_name          47.4
population           36.0
construction_year    34.9
gps_height           34.4
installer             7.5
funder                7.4
scheme_management     6.5
public_meeting        5.6
permit                5.1
longitude             3.1
subvillage            0.6
payment_type          0.0
management_group      0.0
dtype: float64

### Construction Year Imputation

Since `extraction_type_group` is associated with `construction_year`, that will help to impute the values.

Solution by BrendaLoznik

In [6]:
#because of the wide spread of construction years, I don't want to impude by the overall mean
train_df['construction_year'].min(),  train_df['construction_year'].max() , train_df['construction_year'].mean()

(1960.0, 2013.0, 1996.8146855857951)

In [7]:
#We can see that the mean construction year by extraction type group gives much more detailed information
mean_construction = train_df.groupby('extraction_type_group')['construction_year'].mean().reset_index()
mean_construction 

,extraction_type_group,construction_year
0,afridev,2002.316821
1,gravity,1995.965949
2,india mark ii,2001.308428
3,india mark iii,2004.126984
4,mono,1992.634541
5,nira/tanira,1999.421166
6,other,1993.136859
7,other handpump,2000.711790
8,other motorpump,2011.666667
9,rope pump,2005.810256


In [8]:
#create train_df with the mean extraction year by extraction type group
mean_construction = train_df.groupby('extraction_type_group')['construction_year'].mean().reset_index()
mean_construction  = mean_construction .rename(columns={"construction_year": "imputed_construction_year"})

#merge this train_df to the main train_df and replace missing values
train_df = train_df.merge(mean_construction, how =  'left', on =  'extraction_type_group')
train_df['construction_year_imputed'] = np.where(train_df['construction_year'].isna(), train_df['imputed_construction_year'], train_df['construction_year'] )

#drop redundant columns
train_df=train_df.drop(['imputed_construction_year', 'construction_year'], axis=1)

### GPS Height Imputation

Replace values randomly within 1std of the mean, following a normal distribution.

This probably needs verifying/improving but it'll do for now.

Solution by BrendaLoznik

In [9]:
#create mean on the lowest granularity level (subvillage)
means_altitude_subvillage = train_df.groupby(['region', 'lga', 'ward', 'subvillage'])['gps_height'].mean().reset_index()#
means_altitude_subvillage = means_altitude_subvillage.rename(columns={"gps_height": "gps_height_imputed_subvillage"})

#ward level
means_altitude_ward = train_df.groupby(['region', 'lga', 'ward',])['gps_height'].mean().reset_index()
means_altitude_ward = means_altitude_ward.rename(columns={"gps_height": "gps_height_imputed_ward"})

#lga level
means_altitude_lga = train_df.groupby(['region', 'lga'])['gps_height'].mean().reset_index()
means_altitude_lga = means_altitude_lga .rename(columns={"gps_height": "gps_height_imputed_lga"})

#region level
means_altitude_region = train_df.groupby(['region'])['gps_height'].mean().reset_index()
means_altitude_region = means_altitude_region.rename(columns={"gps_height": "gps_height_imputed_region"})

#region basin
means_altitude_basin = train_df.groupby(['basin'])['gps_height'].mean().reset_index()
means_altitude_basin = means_altitude_basin.rename(columns={"gps_height": "gps_height_imputed_basin"})

#merge the aggregated dataframes as new columns to the original train_df
train_df= train_df.merge(means_altitude_subvillage, how = 'left', on = ['region', 'lga', 'ward', 'subvillage'])
train_df = train_df.merge(means_altitude_ward, how = 'left', on = ['region', 'lga', 'ward'])
train_df = train_df.merge(means_altitude_lga, how = 'left', on = ['region', 'lga'])
train_df = train_df.merge(means_altitude_region, how = 'left', on = ['region'])
train_df = train_df.merge(means_altitude_basin, how = 'left', on = ['basin'])

#create final imputed longitude column
train_df['imputed_gps_height'] = np.where(train_df['gps_height'].isna(), train_df['gps_height_imputed_subvillage'], train_df['gps_height']) #if longitude is missing, impute it by the mean of the subvillage
train_df['imputed_gps_height'] = np.where(train_df['imputed_gps_height'].isna(), train_df['gps_height_imputed_ward'], train_df['imputed_gps_height']) #if subvillage mean is missing, impute it by the ward
train_df['imputed_gps_height'] = np.where(train_df['imputed_gps_height'].isna(), train_df['gps_height_imputed_lga'], train_df['imputed_gps_height'])
train_df['imputed_gps_height'] = np.where(train_df['imputed_gps_height'].isna(), train_df['gps_height_imputed_region'], train_df['imputed_gps_height'])
train_df['imputed_gps_height'] = np.where(train_df['imputed_gps_height'].isna(), train_df['gps_height_imputed_basin'], train_df['imputed_gps_height'])

#drop redundant columns
train_df= train_df.drop(['gps_height_imputed_subvillage','gps_height_imputed_ward' , 'gps_height_imputed_lga' , 'gps_height_imputed_region', 'gps_height', 'gps_height_imputed_basin'], axis=1)


### Latitude and Longitude Imputation
0 values are replaced by subvillage, ward, lga, or region's mean logitude and latitude values for each column. 

Solution by BrendaLoznik

In [10]:
train_df['longitude'].replace(0, np.nan, inplace=True)
#create mean longitude on the lowest granularity level (subvillage)
means_longitude_subvillage = train_df.groupby(['region', 'lga', 'ward', 'subvillage'])['longitude'].mean().reset_index()
means_longitude_subvillage = means_longitude_subvillage.rename(columns={"longitude": "longitude_imputed_subvillage"})

#ward level
means_longitude_ward = train_df.groupby(['region', 'lga', 'ward',])['longitude'].mean().reset_index()
means_longitude_ward = means_longitude_ward.rename(columns={"longitude": "longitude_imputed_ward"})

#lga level
means_longitude_lga = train_df.groupby(['region', 'lga'])['longitude'].mean().reset_index()
means_longitude_lga = means_longitude_lga .rename(columns={"longitude": "longitude_imputed_lga"})

#region level
means_longitude_region = train_df.groupby(['region'])['longitude'].mean().reset_index()
means_longitude_region = means_longitude_region.rename(columns={"longitude": "longitude_imputed_region"})
means_longitude_region.head()

,region,longitude_imputed_region
0,Arusha,36.552713
1,Dar es Salaam,39.215799
2,Dodoma,36.044171
3,Iringa,34.895989
4,Kagera,31.233262


In [11]:
#merge the aggregated dataframes as new columns to the original df
train_df= train_df.merge(means_longitude_subvillage, how = 'left', on = ['region', 'lga', 'ward', 'subvillage'])
train_df= train_df.merge(means_longitude_ward, how = 'left', on = ['region', 'lga', 'ward'])
train_df = train_df.merge(means_longitude_lga, how = 'left', on = ['region', 'lga'])
train_df = train_df.merge(means_longitude_region, how = 'left', on = ['region'])

#select the right longitude level based on the availability of information
train_df['imputed_longitude'] = np.where(train_df['longitude'].isna(), train_df['longitude_imputed_subvillage'], train_df['longitude']) #if longitude is missing, impute it by the mean of the subvillage
train_df['imputed_longitude'] = np.where(train_df['imputed_longitude'].isna(), train_df['longitude_imputed_ward'], train_df['imputed_longitude']) #if subvillage mean is missing, impute it by the ward
train_df['imputed_longitude'] = np.where(train_df['imputed_longitude'].isna(), train_df['longitude_imputed_lga'], train_df['imputed_longitude'])
train_df['imputed_longitude'] = np.where(train_df['imputed_longitude'].isna(), train_df['longitude_imputed_region'], train_df['imputed_longitude'])

#drop redundant columns
train_df= train_df.drop(['longitude_imputed_subvillage','longitude_imputed_ward' , 'longitude_imputed_lga' , 'longitude_imputed_region', 'longitude'], axis=1)

In [12]:
train_df['latitude'].where(train_df["latitude"] <= -0.5, np.nan, inplace=True)
#create mean latitude on the lowest granularity level (subvillage)
means_latitude_subvillage = train_df.groupby(['region', 'lga', 'ward', 'subvillage'])['latitude'].mean().reset_index()
means_latitude_subvillage = means_latitude_subvillage.rename(columns={"latitude": "latitude_imputed_subvillage"})

#ward level
means_latitude_ward = train_df.groupby(['region', 'lga', 'ward',])['latitude'].mean().reset_index()
means_latitude_ward = means_latitude_ward.rename(columns={"latitude": "latitude_imputed_ward"})

#lga level
means_latitude_lga = train_df.groupby(['region', 'lga'])['latitude'].mean().reset_index()
means_latitude_lga = means_latitude_lga .rename(columns={"latitude": "latitude_imputed_lga"})

#region level
means_latitude_region = train_df.groupby(['region'])['latitude'].mean().reset_index()
means_latitude_region = means_latitude_region.rename(columns={"latitude": "latitude_imputed_region"})
means_latitude_region.head()

,region,latitude_imputed_region
0,Arusha,-3.246455
1,Dar es Salaam,-6.909677
2,Dodoma,-5.928734
3,Iringa,-8.907700
4,Kagera,-1.961466


In [13]:
#merge the aggregated dataframes as new columns to the original df
train_df= train_df.merge(means_latitude_subvillage, how = 'left', on = ['region', 'lga', 'ward', 'subvillage'])
train_df= train_df.merge(means_latitude_ward, how = 'left', on = ['region', 'lga', 'ward'])
train_df = train_df.merge(means_latitude_lga, how = 'left', on = ['region', 'lga'])
train_df = train_df.merge(means_latitude_region, how = 'left', on = ['region'])

#select the right latitude level based on the availability of information
train_df['imputed_latitude'] = np.where(train_df['latitude'].isna(), train_df['latitude_imputed_subvillage'], train_df['latitude']) #if longitude is missing, impute it by the mean of the subvillage
train_df['imputed_latitude'] = np.where(train_df['imputed_latitude'].isna(), train_df['latitude_imputed_ward'], train_df['imputed_latitude']) #if subvillage mean is missing, impute it by the ward
train_df['imputed_latitude'] = np.where(train_df['imputed_latitude'].isna(), train_df['latitude_imputed_lga'], train_df['imputed_latitude'])
train_df['imputed_latitude'] = np.where(train_df['imputed_latitude'].isna(), train_df['latitude_imputed_region'], train_df['imputed_latitude'])

#drop redundant columns
train_df= train_df.drop(['latitude_imputed_subvillage','latitude_imputed_ward' , 'latitude_imputed_lga' , 'latitude_imputed_region', 'latitude'], axis=1)



### Permit Imputation
Impute missing values with mode of similar records. `management_group` and `public_meeting` were used by BrendaLoznik.

In [14]:
#impute by mode
permit_mg_mode= train_df.groupby(['public_meeting', 'management_group'])['permit'].agg(pd.Series.mode).reset_index()
permit_mg_mode  = permit_mg_mode.rename(columns={"permit": "imputed_permit_mg"})
train_df = train_df.merge(permit_mg_mode, how = 'left', on = ['public_meeting', 'management_group'])

train_df['imputed_permit'] = np.where(train_df['permit'].isna(), train_df['imputed_permit_mg'], train_df['permit'])  #if permit is missing, replace it by the mode of public meeting - management group
train_df['imputed_permit']  = np.where(train_df['imputed_permit'] .isna(), train_df['permit'].mode(), train_df['imputed_permit'])  #if eitther public meeting or management group is missing, then use the mode of permit (True)

#drop original permit column
train_df = train_df.drop(['permit', 'imputed_permit_mg'], axis=1)

### Public Meeting Imputation
Simply impute public_meeting with the mode, as 90% of pumps have a TRUE value

In [15]:
public_meeting_mode = train_df['public_meeting'].mode()[0]
train_df['public_meeting'] = train_df['public_meeting'].fillna(public_meeting_mode)

### Scheme Management Imputation

In [16]:
# it seems like a logical choice to impute missing scheme management values by the mode of the management - scheme-management as there is a lot of overlap here
check = train_df.groupby([  'management_group', 'scheme_management' , 'management'])['id'].count().reset_index()
check = check.sort_values('management')
check.head(10)

scheme_mode = train_df.groupby(['management'])['scheme_management'].agg(pd.Series.mode).reset_index()
scheme_mode = scheme_mode.rename(columns={"scheme_management": "imputed_scheme_management"})
scheme_mode 

#merge scheme_mode to original df and use it to replace missing values
train_df = train_df.merge(scheme_mode, how = 'left', on = [ 'management'])
raw['imputed_scheme__management'] = np.where(raw['scheme_management'].isna(), raw['imputed_scheme_management'], raw['scheme_management'])

#drop redundant columns
raw= raw.drop(['scheme_management', 'imputed_scheme_management'],axis=1)



NameError: name 'raw' is not defined

### Installer

In [ ]:
#I noticed a lot of variation in captialization, so I will first convert al entries to lowercase
raw['installer'] = raw['installer'].str.lower()

#plot top 10 installers
installer = raw.groupby('installer')['id'].count().reset_index()
installer = installer.sort_values('id', ascending = False)
installer.head(10)


#there are a few categories in the top 60 most common categories whose naims look a lot alike and are probably typo's. We will merge them together
raw['installer'] = np.where( raw['installer']=='gove', 'gover', raw['installer'] )
raw['installer'] = np.where( raw['installer']=='community', 'commu', raw['installer'] )
raw['installer'] = np.where( raw['installer']=='danid', 'danida', raw['installer'] )

installer = raw.groupby('installer')['id'].count().reset_index()
installer = installer.sort_values('id', ascending = False)
installer.head(10)

#I want to keep the most frequent installers and combine the rarer classes together. I have played around with the optimum number of installers to keep, and I decided on the top 150.

#create list of top 150 installers
top_installers = installer.nlargest(150, 'id')['installer'].unique()

#replace funders that are not in top 10 with 'other'
raw['installer'] = np.where(raw['installer'].isin(top_installers), raw['installer'], 'other')


### Funder

In [ ]:
#set all entries to lowercase
train_df['funder'] = raw['funder'].str.lower()


funder = train_df.groupby('funder')['id'].count().reset_index()
funder = funder.sort_values('id', ascending = False)
funder.head(10)

#create list of top 150 funders
top_funders = funder.nlargest(150, 'id')['funder'].unique()

#replace funders that are not in top 150 with 'other'
train_df['funder'] = np.where(train_df['funder'].isin(top_funders), train_df['funder'], 'other')


### Population
Brenda's solution considers region-wide population average when not available, which is a poor indicator of population. Some readings also have very large population values, which skew the average. We use a binning approach to better represent population, with a "missing" bin where subvillage or ward population is unknown, as this itself is a decent predictor.

NameError: name 'sns' is not defined

## 2. Feature Engineering

### recorded_age Feature
Create a feature recorded_age, which states the age (in years) of the pump at time of recording. Uses date_recorded and construction_year.

In [ ]:
train_df['recorded_year'] = pd.DatetimeIndex(train_df['date_recorded']).year
train_df['recorded_age'] = train_df['recorded_year'] - train_df['construction_year']
train_df = train_df.drop('recorded_year',axis=1)

## 3. (Partial) Feature Selection
Remove bad features here.

### Columns to Drop
amount_tsh: *70% missing values*

date_recorded: *Used in recorded_age engineered feature*



In [ ]:
train_df = train_df.drop(["date_recorded", "scheme_name"])